In [1]:
!pip install -U bitsandbytes

In [2]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_DEFAULT_REGION'] = ''  # e.g., 'us-east-1'

In [ ]:
!pip install boto3
import boto3
import os
import json
!pip install mistralai
from mistralai import Mistral
mistral_api_key = ''
client = Mistral(api_key=mistral_api_key)
s3 = boto3.client('s3')
bucket_name = 'hydroponic123'
prefix = 'good-strawberry1.jpg'
client = Mistral(api_key=mistral_api_key)

In [5]:
def list_images_in_s3(bucket, prefix=''):
    """List image files in an S3 bucket with a prefix"""
    response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
    return [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].lower().endswith(('.jpg', '.png'))]

def get_presigned_url(bucket, key, expiration=3600):
    """Generate a temporary URL to access the S3 object"""
    return s3.generate_presigned_url('get_object',
                                     Params={'Bucket': bucket, 'Key': key},
                                     ExpiresIn=expiration)


In [6]:
def analyze_image(image_url):
    """Send image to Mistral API for analysis"""
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": 'Analyze this strawberry plant image in detail and provide:\n'
                            '1. Overall plant health status\n'
                            '2. Any signs of fungal diseases\n'
                            '3. Fruit condition (if visible)\n'
                            '4. Leaf health\n'
                            '5. Particular diseases_detected (if any)\n'
                            '6. Recommendations to decrease the disease spread in hydroponic farm\n'
                            '7. Give me the particular fungiside or any that will decrease the disease\n'
                            '8. How to use the fungisides process in hydroponic farm\n'
                            '6. Recommended actions\n'
                            'Format the response as a JSON object with these keys:\n'
                            'health_status, fungal_status, fruit_condition, leaf_health,\n'
                            'diseases_detected, recommendations'
                },
                {
                    "type": "image_url",
                    "image_url": image_url
                }
            ]
        }
    ]

    try:
        chat_response = client.chat.complete(
            model="pixtral-12b-2409",
            messages=messages
        )
        return chat_response.choices[0].message.content
    except Exception as e:
        return {"error": str(e)}


In [7]:

if __name__ == "__main__":
    image_keys = list_images_in_s3(bucket_name, prefix)

    results = []

    for key in image_keys:
        file_name = key.split("/")[-1]
        image_url = get_presigned_url(bucket_name, key)

        print(f"Analyzing image: {file_name}")
        analysis_result = analyze_image(image_url)

        results.append({
            "file_name": file_name,
            "s3_key": key,
            "analysis_result": analysis_result
        })
        print(f"Analysis complete for {file_name}")
        print("-" * 50)

    with open("image_analysis_results.json", "w") as f:
        json.dump(results, f, indent=4)

    print("Results written to image_analysis_results.json")


Analyzing image: good-strawberry1.jpg
Analysis complete for good-strawberry1.jpg
--------------------------------------------------
Results written to image_analysis_results.json


In [8]:
with open("image_analysis_results.json", "r") as f:
    data = json.load(f)

print(json.dumps(data, indent=4))

[
    {
        "file_name": "good-strawberry1.jpg",
        "s3_key": "good-strawberry1.jpg",
        "analysis_result": "```json\n{\n  \"health_status\": \"The overall plant health appears to be good. The strawberries are vibrant and the leaves are green and healthy.\",\n  \"fungal_status\": \"No visible signs of fungal diseases on the strawberries or leaves.\",\n  \"fruit_condition\": \"The fruits are ripe, red, and appear to be in good condition without any visible blemishes or deformities.\",\n  \"leaf_health\": \"The leaves are green and appear healthy with no visible signs of disease or pests.\",\n  \"diseases_detected\": \"No particular diseases detected.\",\n  \"recommendations\": {\n    \"preventive_measure\": \"To maintain plant health and prevent potential fungal diseases, consider the following measures in a hydroponic farm:\",\n    \"environmental_control\": \"Maintain proper humidity and temperature levels to reduce the risk of fungal infections.\",\n    \"sanitation\": 

In [11]:
%pip install sentence-transformers

%pip install faiss-cpu

from mistralai import Mistral

from sentence_transformers import SentenceTransformer
import faiss
import json
import requests

# Specify model
model = "pixtral-12b-2409"

knowledge_base = [
    "Powdery mildew causes white fuzzy spots on strawberry leaves in humid environments.",
    "Leaf spot shows up as brown or purple lesions on the leaf surface.",
    "Root rot causes wilting and yellowing due to poor oxygen supply in hydroponic water.",
    "Chlorosis (yellowing leaves) may indicate nutrient deficiencies or water stress.",
    "Stunted growth could be caused by root problems or nutrient imbalances.",
    "Nutrient deficiencies (e.g., nitrogen) can cause yellowing of the leaves.",
    "Calcium deficiency causes malformed fruit and leaf tip burn.",
    "Algae growth in hydroponic systems can clog pipes and affect nutrient absorption.",
    "Anthracnose causes **dark sunken lesions on leaves** and **small brown, water-soaked spots** on fruits with **orange spore masses in humid conditions**.",
    "Gray mold (Botrytis cinerea) appears as **fuzzy gray and fuzzy white mold** mainly on **ripening or damaged fruits**, especially in **cool, damp environments**.",
    "Botrytis affects fruits and flowers, forming **gray-white mold patches** that are **soft and mushy to the touch**, often appearing **after harvest or injury**.",
    # Add more knowledge entries here
]



# Step 1: Create embeddings for knowledge base using SentenceTransformer
embedder = SentenceTransformer("all-MiniLM-L6-v2")
kb_embeddings = embedder.encode(knowledge_base)

# Step 2: Set up FAISS index for retrieval
index = faiss.IndexFlatL2(kb_embeddings.shape[1])
index.add(kb_embeddings)

# Step 3: Simulated image query (you could replace this with actual image analysis)
query = "white fuzzy spots on strawberry leaves"
query_embedding = embedder.encode([query])
_, top_indices = index.search(query_embedding, k=2)  # Top 2 relevant documents
retrieved_docs = [knowledge_base[i] for i in top_indices[0]]
rag_context = "\n".join(retrieved_docs)

# Step 4: Define the messages for the chat
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": f"""Analyze this strawberry plant image in detail. Use the following context for disease detection:

{rag_context}

Return a JSON object with the following keys:
- health_status
- fungal_status
- fruit_condition
- leaf_health
- particular diseases_detected
- Recommendations to decrease the disease spread in hydroponic farm if health_status is good please provide some to strengthen the plant and maintain the health
- Give me the particular fungiside or any that will decrease the disease if infected else return not needed
- How to use the fungisides process in hydroponic farm"""
            },
            {
                "type": "image_url",
                "image_url": image_url
            }
        ]
    }
]

# Step 5: Get chat response
try:
    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )

    # Print the content of the response
    print(chat_response.choices[0].message.content)

except Exception as e:
    print(f"Error occurred: {str(e)}")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.
```json
{
  "health_status": "Good",
  "fungal_status": "No visible signs of fungal infection",
  "fruit_condition": "Excellent, no visible damage or mold",
  "leaf_health": "Good, no visible signs of powdery mildew",
  "particular_diseases_detected": "None",
  "Recommendations": [
    "Maintain proper air circulation around the plants.",
    "Ensure the hydroponic system is clean and free of debris.",
    "Use beneficial bacteria and fungi to promote plant health.",
    "Monitor humidity levels to prevent excessive moisture.",
    "Regularly inspect plants for early signs of disease."
  ],
  "fungicide_recommendation": "Not needed",
  "fungicide_usage_instructions": "N/A"
}
```


In [ ]:
# Example: List all objects in the bucket
response = s3.list_objects_v2(Bucket=bucket_name)
if 'Contents' in response:
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("No objects found in the bucket.")

bad-strawberry1.jpg
bad-strawberry10.jpg
bad-strawberry19.jpg
bad-strawberry3.jpg
good-strawberry1.jpg
